---

In [4]:
import mlflow
import sys
import os

sys.path.append("../../")
from ML.utils.mlflow_flow import set_tracking
from dotenv import load_dotenv

load_dotenv()
ENDPOINT_URL = os.getenv("ENDPOINT_URL")
DAGSHUB_PUBLIC_ACCESS_KEY = os.getenv("DAGSHUB_PUBLIC_ACCESS_KEY")

os.unsetenv("MLFLOW_TRACKING_URI")
os.unsetenv("MLFLOW_REGISTRY_URI")
os.unsetenv("MLFLOW_AUTH_CONFIG_PATH")

os.environ["MLFLOW_TRACKING_USERNAME"] = "Yosesotomayor"
os.environ["MLFLOW_TRACKING_PASSWORD"] = DAGSHUB_PUBLIC_ACCESS_KEY


---

In [2]:
sys.path.append("../models/ensemble_elnet_lgbm")

MODEL_NAME = 'elnet_lgbm'
set_tracking(ENDPOINT_URL)

model_r = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@champion")


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/alias failed with error code 401 != 200. Response body: ''

---